In [ ]:
# default_exp funcs.predict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from decode_fish.imports import *
from decode_fish.funcs.utils import *
from monai.inferers import sliding_window_inference
from decode_fish.funcs.emitter_io import append_emitter_df

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_MOp/sweep_mop_19/norm:escortxz_facs:Truexmean_diff:Truexlr:0.0005//train.yaml')

[autoreload of decode_fish.engine.place_psfs failed: Traceback (most recent call last):
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in u

In [ ]:
from decode_fish.funcs.file_io import load_all
model, post_proc, micro, img_3d, decode_dl = load_all(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 56, 56)
7 volumes


/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# export
def window_predict(model, post_proc, image_vol, window_size=[None,256,256], crop=np.s_[:,:,:,:,:], bs=1, device='cuda', chrom_map=None, scale=None, progress_bar=False):
    pred_df = DF()
    with torch.no_grad():
                
        print(image_vol.shape)
        if image_vol.ndim == 4:
            image_vol = image_vol[None]
            
        n_batches = int(np.ceil(len(image_vol)/bs))

        if scale is not None:
            image_vol = image_vol * scale.to(image_vol.device)

        if chrom_map is not None:
            image_vol = torch.concat([image_vol,chrom_map.to(image_vol.device).repeat_interleave(len(image_vol),0)], 1)

        if crop is not None:
            image_vol = image_vol[crop]
        
        for i in tqdm(range(n_batches), disable=not(progress_bar)):
            inp = image_vol[i*bs:(i+1)*bs]
            output = sliding_window_inference(inp, window_size, 1, model.to(device), overlap=0.2, sw_device=device, device='cpu', mode='gaussian')
            output = model.tensor_to_dict(output)
            p_si = sliding_window_inference(output['logits'], window_size, 1, post_proc, overlap=0.2, sw_device=device, device='cpu', mode='gaussian')
            i_df = post_proc.get_df(output, p_si)
            pred_df = append_emitter_df(pred_df, i_df)
            free_mem()
        
        return pred_df

In [ ]:
res_df = window_predict(model, post_proc, decode_dl.dataset.volumes, window_size=[None, 128, 128], crop=np.s_[:,:,:,500:1500,500:1500], device='cuda', chrom_map=get_color_shift_inp(micro.color_shifts, micro.col_shifts_yx)[:,:,None], scale=micro.get_ch_mult())

torch.Size([7, 22, 1, 2048, 2048])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:23<00:00, 11.91s/it]


In [ ]:
res_df

,loc_idx,frame_idx,code_inds,x,y,z,prob,x_sig,y_sig,z_sig,...,int_17,int_sig_17,int_18,int_sig_18,int_19,int_sig_19,int_20,int_sig_20,int_21,int_sig_21
0,0,0,1,74171.859375,41130.691406,26.101784,0.543179,15.858923,14.163632,10.162832,...,2.039867,0.410154,3.554771,0.424439,1.140061,0.226346,0.897971,0.254912,2.315177,0.306522
1,1,0,1,96872.078125,43359.386719,82.825485,0.505354,13.718910,24.272238,8.824762,...,2.107101,0.521238,0.985963,0.373564,0.416336,0.385557,0.498232,0.287192,4.882941,1.008928
2,2,0,1,27739.687500,51608.562500,78.598030,0.635778,25.854805,26.916786,23.552073,...,0.386799,0.474805,0.369928,0.384921,0.961122,0.961948,0.527913,0.503030,4.809768,1.074089
3,3,0,1,30754.794922,51744.761719,78.132843,0.523369,31.643612,24.412115,16.417786,...,0.738403,0.519148,1.289219,0.771810,2.058728,0.790543,1.411235,0.720317,6.489904,2.228016
4,4,0,2,19370.750000,35763.464844,59.256752,0.750923,16.137478,17.508348,16.802031,...,0.709408,0.474716,0.475889,0.369654,0.932401,0.575597,0.417964,0.362533,2.991047,0.528999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,11846,6,240,26360.894531,54250.007812,49.699955,0.688423,17.920855,17.689392,23.722155,...,0.410511,0.320712,0.383306,0.202143,0.497893,0.314297,0.154870,0.312463,0.175402,0.268297
11847,11847,6,240,26846.646484,56829.156250,22.386724,0.998641,9.082379,11.517522,9.780725,...,0.225731,0.165710,0.612131,0.209014,0.362049,0.190732,0.348656,0.164641,0.370926,0.231139
11848,11848,6,241,63397.339844,30798.437500,53.324135,1.000000,5.773283,9.551782,7.292207,...,5.375982,0.787546,0.787245,0.279420,0.615147,0.226852,4.074576,0.527864,0.624959,0.216693
11849,11849,6,249,11280.794922,12305.125000,63.883244,0.564273,28.645763,25.192028,15.955474,...,0.778318,0.460973,1.964516,0.776545,2.120476,0.816587,0.940884,0.488861,1.465305,0.451530


In [ ]:
from monai.data.utils import compute_importance_map, dense_patch_slices, get_valid_patch_size
from monai.utils import BlendMode, PytorchPadMode, fall_back_tuple
from typing import Any, Callable, List, Sequence, Tuple, Union
import torch.nn.functional as F
__all__ = ["sliding_window_inference"]


def sliding_window_inference(
    inputs: torch.Tensor,
    roi_size: Union[Sequence[int], int],
    sw_batch_size: int,
    predictor: Callable[..., torch.Tensor],
    overlap: float = 0.25,
    threshold: float = -1,
    mode: Union[BlendMode, str] = BlendMode.CONSTANT,
    sigma_scale: Union[Sequence[float], float] = 0.125,
    padding_mode: Union[PytorchPadMode, str] = PytorchPadMode.CONSTANT,
    cval: float = 0.0,
    sw_device: Union[torch.device, str, None] = None,
    device: Union[torch.device, str, None] = None,
    *args: Any,
    **kwargs: Any,
) -> torch.Tensor:
    """
    Sliding window inference on `inputs` with `predictor`.

    When roi_size is larger than the inputs' spatial size, the input image are padded during inference.
    To maintain the same spatial sizes, the output image will be cropped to the original input size.

    Args:
        inputs: input image to be processed (assuming NCHW[D])
        roi_size: the spatial window size for inferences.
            When its components have None or non-positives, the corresponding inputs dimension will be used.
            if the components of the `roi_size` are non-positive values, the transform will use the
            corresponding components of img size. For example, `roi_size=(32, -1)` will be adapted
            to `(32, 64)` if the second spatial dimension size of img is `64`.
        sw_batch_size: the batch size to run window slices.
        predictor: given input tensor `patch_data` in shape NCHW[D], `predictor(patch_data)`
            should return a prediction with the same spatial shape and batch_size, i.e. NMHW[D];
            where HW[D] represents the patch spatial size, M is the number of output channels, N is `sw_batch_size`.
        overlap: Amount of overlap between scans.
        mode: {``"constant"``, ``"gaussian"``}
            How to blend output of overlapping windows. Defaults to ``"constant"``.

            - ``"constant``": gives equal weight to all predictions.
            - ``"gaussian``": gives less weight to predictions on edges of windows.

        sigma_scale: the standard deviation coefficient of the Gaussian window when `mode` is ``"gaussian"``.
            Default: 0.125. Actual window sigma is ``sigma_scale`` * ``dim_size``.
            When sigma_scale is a sequence of floats, the values denote sigma_scale at the corresponding
            spatial dimensions.
        padding_mode: {``"constant"``, ``"reflect"``, ``"replicate"``, ``"circular"``}
            Padding mode for ``inputs``, when ``roi_size`` is larger than inputs. Defaults to ``"constant"``
            See also: https://pytorch.org/docs/stable/nn.functional.html#pad
        cval: fill value for 'constant' padding mode. Default: 0
        sw_device: device for the window data.
            By default the device (and accordingly the memory) of the `inputs` is used.
            Normally `sw_device` should be consistent with the device where `predictor` is defined.
        device: device for the stitched output prediction.
            By default the device (and accordingly the memory) of the `inputs` is used. If for example
            set to device=torch.device('cpu') the gpu memory consumption is less and independent of the
            `inputs` and `roi_size`. Output is on the `device`.
        args: optional args to be passed to ``predictor``.
        kwargs: optional keyword args to be passed to ``predictor``.

    Note:
        - input must be channel-first and have a batch dim, supports N-D sliding window.

    """
    num_spatial_dims = len(inputs.shape) - 2
    if overlap < 0 or overlap >= 1:
        raise AssertionError("overlap must be >= 0 and < 1.")

    # determine image spatial size and batch size
    # Note: all input images must have the same image size and batch size
    image_size_ = list(inputs.shape[2:])
    batch_size = inputs.shape[0]

    if device is None:
        device = inputs.device
    if sw_device is None:
        sw_device = inputs.device

    roi_size = fall_back_tuple(roi_size, image_size_)
    # in case that image size is smaller than roi size
    image_size = tuple(max(image_size_[i], roi_size[i]) for i in range(num_spatial_dims))
    pad_size = []
    for k in range(len(inputs.shape) - 1, 1, -1):
        diff = max(roi_size[k - 2] - inputs.shape[k], 0)
        half = diff // 2
        pad_size.extend([half, diff - half])
    inputs = F.pad(inputs, pad=pad_size, mode=PytorchPadMode(padding_mode).value, value=cval)

    scan_interval = _get_scan_interval(image_size, roi_size, num_spatial_dims, overlap)

    # Store all slices in list
    slices = dense_patch_slices(image_size, roi_size, scan_interval)
    num_win = len(slices)  # number of windows per image
    total_slices = num_win * batch_size  # total number of windows

    # Create window-level importance map
    importance_map = compute_importance_map(
        get_valid_patch_size(image_size, roi_size), mode=mode, sigma_scale=sigma_scale, device=device
    )
    window_output_shape = None
    counter = 0

    # Perform predictions
    output_image, count_map = torch.tensor(0.0, device=device), torch.tensor(0.0, device=device)
    _initialized = False
    for slice_g in range(0, total_slices, sw_batch_size):
        slice_range = range(slice_g, min(slice_g + sw_batch_size, total_slices))
        unravel_slice = [
            [slice(int(idx / num_win), int(idx / num_win) + 1), slice(None)] + list(slices[idx % num_win])
            for idx in slice_range
        # ]
        window_data = torch.cat([inputs[win_slice] for win_slice in unravel_slice]).to(sw_device)
        if window_data.max() > threshold or window_output_shape is None:
            seg_prob = predictor(window_data, *args, **kwargs).to(device)  # batched patch segmentation
            window_output_shape = seg_prob.shape
            counter += 1
        else:
            seg_prob = torch.zeros(window_output_shape) - 100

        if not _initialized:  # init. buffer at the first iteration
            output_classes = seg_prob.shape[1]
            output_shape = [batch_size, output_classes] + list(image_size)
            # allocate memory to store the full output and the count for overlapping parts
            output_image = torch.zeros(output_shape, dtype=torch.float32, device=device)
            count_map = torch.zeros(output_shape, dtype=torch.float32, device=device)
            _initialized = True

        # store the result in the proper location of the full output. Apply weights from importance map.
        for idx, original_idx in zip(slice_range, unravel_slice):
            output_image[original_idx] += importance_map * seg_prob[idx - slice_g]
            count_map[original_idx] += importance_map

    # account for any overlapping sections
    output_image = output_image / count_map

    final_slicing: List[slice] = []
    for sp in range(num_spatial_dims):
        slice_dim = slice(pad_size[sp * 2], image_size_[num_spatial_dims - sp - 1] + pad_size[sp * 2])
        final_slicing.insert(0, slice_dim)
    while len(final_slicing) < len(output_image.shape):
        final_slicing.insert(0, slice(None))
        
    print(counter)
    return output_image[final_slicing]


def _get_scan_interval(
    image_size: Sequence[int], roi_size: Sequence[int], num_spatial_dims: int, overlap: float
) -> Tuple[int, ...]:
    """
    Compute scan interval according to the image size, roi size and overlap.
    Scan interval will be `int((1 - overlap) * roi_size)`, if interval is 0,
    use 1 instead to make sure sliding window works.

    """
    if len(image_size) != num_spatial_dims:
        raise ValueError("image coord different from spatial dims.")
    if len(roi_size) != num_spatial_dims:
        raise ValueError("roi coord different from spatial dims.")

    scan_interval = []
    for i in range(num_spatial_dims):
        if roi_size[i] == image_size[i]:
            scan_interval.append(int(roi_size[i]))
        else:
            interval = int(roi_size[i] * (1 - overlap))
            scan_interval.append(interval if interval > 0 else 1)
    return tuple(scan_interval)

In [ ]:
def selective_predict(model, post_proc, image_paths, window_size=[None, 128, 128], threshold=-1e6, device='cuda'):
    pred_df = DF()
    with torch.no_grad():
        for p in tqdm(image_paths):
            print(p.split('/')[-1])
            img = load_tiff_image(p)
            z,y,x = img.shape[-3:]
            img = img.reshape(-1,z,y,x)
            for i in range(len(img)):
                print(img[i][None,None].shape)
                output = sliding_window_inference(img[i][None,None], window_size, sw_batch_size=1, predictor=model.to(device), overlap=0.2, threshold=threshold, sw_device=device, device='cpu', mode='gaussian')
                # output = model.cpu()(img[i][None,None])
                output = model.tensor_to_dict(output)
                p_si = sliding_window_inference(output['logits'], window_size, sw_batch_size=1, predictor=post_proc, overlap=0.2, threshold=-5, sw_device=device, device='cpu', mode='gaussian')
                i_df = post_proc.get_df(output, p_si)
                print('N. emitters: ', len(i_df))
                pred_df = append_emitter_df(pred_df, i_df)
                free_mem()
        return pred_df

In [ ]:
t0 = time.time()
for _ in range(5):
    pred_df = selective_predict(model, post_proc, image_paths, threshold=0)
print(time.time() - t0)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]

experiment_1_smfish_fov_1.tif
torch.Size([1, 1, 23, 650, 500])
130


  0%|                                                                         | 0/1 [00:00<?, ?it/s]

84
N. emitters:  737
experiment_1_smfish_fov_1.tif
torch.Size([1, 1, 23, 650, 500])
130


  0%|                                                                         | 0/1 [00:00<?, ?it/s]

...

experiment_1_smfish_fov_1.tif
torch.Size([1, 1, 23, 650, 500])
130


  0%|                                                                         | 0/1 [00:00<?, ?it/s]

84
N. emitters:  737
experiment_1_smfish_fov_1.tif
torch.Size([1, 1, 23, 650, 500])
130
84
N. emitters:  737


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.55s/it]

12.249284982681274


In [ ]:
!nbdev_build_lib

[autoreload of decode_fish.engine.place_psfs failed: Traceback (most recent call last):
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in u

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 02b_place_psfs.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted 24_exp_specific.ipynb.
Converted 25_ensembling.ipynb.
Converted 25_gen_train.ipynb.
Converted index.ipynb.
